In [26]:
import numpy as np
from numpy import genfromtxt
from itertools import combinations
import torch
import math 
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
%run r3b_clustering_def.ipynb
%run agglomerative_def.ipynb
%run ff_models_def.ipynb

In [27]:
## TODO: check that you do correctly shifting the time info for the agglo model...
def subset_data_sel(data,sel_type,model,f_out_name):
    if (model == "r3b"):
        reco_vals = run_r3b_clustering(data,0.25)
    elif (model == "agglo"):
        reco_vals = run_agglo_clustering(data,3540)
    else: 
        print("something went wrong, have you chosen the correct model?")
    true_vals = get_true_combinatorics(data)    
    print("now I have all simple stuff done")
    idx = 0
    fmt = '%i','%1.5f', '%1.5f','%1.5f','%1.2f' 
    arr = np.empty((0, 5), float)
    with open(f_out_name, 'w') as f:
        for i in range(0,int(np.max(data,axis=0)[0]+1)-(int(np.max(data,axis=0)[0]+1)%3) ,3):
        #for i in range(0,int(np.max(data,axis=0)[0])-(int(np.max(data,axis=0)[0])%3) ,3):
            if i % 1000 == 0:
                percentage = (i /int(np.max(data,axis=0)[0]+1)) * 100
                print(f"Progress: {percentage:.2f}%")
            mask = ((data[:,0] == i) | (data[:,0] == i+1) | (data[:,0] == i+2))
            evt = data[mask]
            pair_indices = list(combinations(range(evt.shape[0]), 2))
            ##pair indices which belong together...
            pred_comb = []
            ##TJ this is the new part with true stuff
            true_comb = []
            for m in range(evt.shape[0]):
                pred_comb.append((m,m))
                true_comb.append((m,m))
            for j in range (len(pair_indices)):
                if reco_vals[idx+j] > 0.75:
                    pred_comb.append(pair_indices[j])
                if true_vals[idx+j] > 0.75:   
                    true_comb.append(pair_indices[j])
            G = nx.Graph()
            G.add_edges_from(pred_comb)
            connected_components = list(nx.connected_components(G))
            ##TJ again, this it the addition for the true values
            T = nx.Graph()
            T.add_edges_from(true_comb)
            true_components = list(nx.connected_components(T))
            set_true = {frozenset(sublist) for sublist in true_components}
            set_pred = {frozenset(sublist) for sublist in connected_components}
            evt_type =  check_matrix(set_true,set_pred)
            if (evt_type == sel_type):
                for k in range(len(connected_components)):
                    cluster_data = evt[list(connected_components[k]),:]
                    cluster_data_cart = sph2cart(cluster_data)
                    cluster_data_cart_cms = func_cm(cluster_data_cart)
                    cluster_data_sph_cms = cart2sph(cluster_data_cart_cms)
                    cluster_data_sph_cms[:,4] = cluster_data_sph_cms[:,4] -4500
                    arr = np.vstack((arr,cluster_data_sph_cms[0,:].reshape(-1,5)))
            idx += len(pair_indices)
        unique_values, continuous_numbers = np.unique(arr[:,0], return_inverse=True)
        arr[:,0] = continuous_numbers
        np.savetxt(f,arr.reshape(-1,5),fmt = fmt, delimiter=",")
    f.close()
    return f
  
    


In [28]:
def remove_duplicates_from_file(file_path):
    with open(file_path, 'r') as file:
        # Read all lines and remove leading/trailing whitespace
        lines = file.readlines()
    
    # Use a set to eliminate duplicates while preserving order
    unique_lines = list(dict.fromkeys(line.strip() for line in lines))
    
    # Write the unique lines back to the file or a new file
    with open(file_path, 'w') as file:
        for line in unique_lines:
            file.write(line + '\n')

In [31]:
#data = genfromtxt('raw_data_test.txt', delimiter=',')
#data = genfromtxt('final_uniform_validation.txt', delimiter=',')
data = genfromtxt('./sim_data/data_uniform/training_raw.txt', delimiter=',')
#data = genfromtxt('./sim_data/smaller_data_uniform/final_uniform_validation.txt', delimiter=',')
#data = genfromtxt('validation_raw.txt',delimiter=',')
#data = genfromtxt('data_stream_2121.txt', delimiter=',')
data[:,4] = data[:,4]+4500  #this step is needed, I only want positive time values, so that I can use the time as a radius
#my_data[:,4] = 1  #this step is needed, I only want positive time values, so that I can use the time as a radius

# ### structure of mydata : eventnr, energy, theta, phi, hit-time
data = data*[1.,1.,3.14159/180,3.14159/180,1.]
unique_values, continuous_numbers = np.unique(data[:,0], return_inverse=True)
data[:,0] = continuous_numbers
test_reco_v = subset_data_sel(data,"fn","agglo","./sim_data/data_uniform/false_neg_uniform.txt")

38763
Progress: 0.00%
Progress: 7.74%
Progress: 15.48%
Progress: 23.22%
Progress: 30.96%
Progress: 38.69%
Progress: 46.43%
Progress: 54.17%
Progress: 61.91%
Progress: 69.65%
Progress: 77.39%
Progress: 85.13%
Progress: 92.87%
now I have all simple stuff done
Progress: 0.00%
Progress: 7.74%
Progress: 15.48%
Progress: 23.22%
Progress: 30.96%
Progress: 38.69%
Progress: 46.43%
Progress: 54.17%
Progress: 61.91%
Progress: 69.65%
Progress: 77.39%
Progress: 85.13%
Progress: 92.87%


In [5]:
remove_duplicates_from_file('false_neg_uniform.txt')

In [30]:
#data = genfromtxt('raw_data_test.txt', delimiter=',')
#data = genfromtxt('final_uniform_validation.txt', delimiter=',')
#data = genfromtxt('./sim_data/data_uniform/data_raw_uniform_training.csv', delimiter=',')
#data = genfromtxt('./sim_data/data_2_1_mev/training_raw.txt', delimiter=',')
data = genfromtxt('./sim_data/data_uniform/training_raw.txt', delimiter=',')
#data = genfromtxt('validation_raw.txt',delimiter=',')
#data = genfromtxt('data_stream_2121.txt', delimiter=',')
data[:,4] = data[:,4]+4500  #this step is needed, I only want positive time values, so that I can use the time as a radius
#my_data[:,4] = 1  #this step is needed, I only want positive time values, so that I can use the time as a radius

# ### structure of mydata : eventnr, energy, theta, phi, hit-time
data = data*[1.,1.,3.14159/180,3.14159/180,1.]
unique_values, continuous_numbers = np.unique(data[:,0], return_inverse=True)
data[:,0] = continuous_numbers
test_reco_v = subset_data_sel(data,"fn","r3b","./sim_data/data_uniform/false_neg_uniform_r3b.txt")

(93997, 5)
[0.0000e+00 1.0000e+00 2.0000e+00 ... 3.8762e+04 3.8763e+04 3.8764e+04]
something goes completely wrong!!!
angles phi1,2, theta1,2: 2.021229192888889 2.021229192888889 1.331397115361111 1.331397115361111
1.0000000000000002
something goes completely wrong!!!
angles phi1,2, theta1,2: 1.6294257960388887 1.6294257960388887 0.8135286931222222 0.8135286931222222
1.0000000000000002
now I have all simple stuff done
Progress: 0.00%
Progress: 7.74%
Progress: 15.48%
Progress: 23.22%
Progress: 30.96%
Progress: 38.69%
Progress: 46.43%
Progress: 54.17%
Progress: 61.91%
Progress: 69.65%
Progress: 77.39%
Progress: 85.13%
Progress: 92.87%
